In [2]:
!nvidia-smi

Mon Dec 14 21:18:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   27C    P0    27W / 250W |     10MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:0D:00.0 Off |                    0 |
| N/A   28C    P0    28W / 250W |      0MiB / 16280MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [1]:
import torch
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import torchvision.datasets
from torchvision import transforms

import tensorly as tl

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [3]:
PATH = 'CIFAR10_ShuffleNetV2.pth'
#PATH = 'CIFAR10_ResNet18.pth'
model = torch.load(PATH)
model.eval()

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_

In [4]:
from decompositions import cp_decomposition_conv_layer, tucker_decomposition_conv_layer

In [5]:
named_modules = [(name,module) for name, module in model.named_modules() if type(module) not in (nn.Sequential,
                                                torchvision.models.shufflenetv2.ShuffleNetV2,
                                                torch.nn.modules.container.Sequential,
                                                torchvision.models.shufflenetv2.InvertedResidual)                
                            ]
len(named_modules)

151

In [9]:
for _, module in named_modules:
    print(type(module))

<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.pooling.MaxPool2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
<class 'torch.nn.modules.conv.Conv2d

In [7]:
def custom_set_layer(model, name, value):
    split_name = name.split(".")
    layer = model

    for i in range(len(split_name)-1):
        if split_name[i].isnumeric():
            layer = layer[int(split_name[i])]
        else:
            layer = getattr(layer, split_name[i])


    if split_name[-1].isnumeric():
        layer[int(split_name[-1])] = value
    else:
        setattr(layer, split_name[i], value)

In [10]:
for i, (name, module) in enumerate(named_modules):
    print(i, name, module)

0 conv1.0 Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
1 conv1.1 BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
2 conv1.2 ReLU(inplace=True)
3 maxpool MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
4 stage2.0.branch1.0 Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
5 stage2.0.branch1.1 BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
6 stage2.0.branch1.2 Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
7 stage2.0.branch1.3 BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
8 stage2.0.branch1.4 ReLU(inplace=True)
9 stage2.0.branch2.0 Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
10 stage2.0.branch2.1 BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
11 stage2.0.branch2.2 ReLU(inplace=True)
12 stage2.0.branch2.3 Conv2d(58, 58, kernel_size

In [8]:
model.eval()
model.cpu()
tl.set_backend('pytorch')
N = len(named_modules)
for i, (name, module) in enumerate(named_modules):
    print(i, name, module)
    #??????????? first conv layer
    if i >= N - 2 or i == 0:
        break
    
    if isinstance(module, torch.nn.modules.conv.Conv2d):
        conv_layer = module
        if True:
            rank = max(conv_layer.weight.data.numpy().shape)//3
            #print("inside---", i, name, conv_layer)
            decomposed = cp_decomposition_conv_layer(conv_layer, rank) 
        else:
            decomposed = tucker_decomposition_conv_layer(conv_layer)
        
        custom_set_layer(model, name, decomposed)

torch.save(model, 'CIFAR10_ShuffleNetV2_compressed_v1.pth')

0 conv1.0 Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)


In [12]:
i

150